In [1]:
import numpy as np
import math
import tensorflow as tf


In [39]:
def uSoln(u0, ds, inFilt, pFilt, b):
    inChan, row, col = u0.shape
    tMax, outChan = inFilt.shape[0], inFilt.shape[2]
    u = np.zeros((tMax, outChan, row, col))
    x = np.zeros((tMax, outChan, row, col))
    # y=np.zeros((tMax,outChan,row,col))
    bzero = np.zeros((outChan))
    # print('outChan=',outChan,', u shape:',u.shape)

    # print('inFilt shape=',inFilt.shape,', u0 shape:',u0.shape)
    u[0] = conv(u0, inFilt[0], bzero)
    # print('u0',u[0])
    for h in range(tMax - 1):
        print('h=',h)
        x[h] = act(u[h])
        zInt = np.array([conv(x[s], pFilt[s, h], b[h] / ((h + 1) * ds)) for s in range(h + 1)])
        z = trapInt(zInt, ds)
        print("z[with h^]: \n", z)
        u[h + 1] = z + conv(u0, inFilt[h + 1], bzero)
    x[tMax - 1] = act(u[tMax - 1])
    return x, u

In [3]:
def im2col(image, kerRow, kerCol, st):
    """
    Returns:
      col: (new_h*new_w,kerRow*KerCol*imageChan) matrix,
            each column is a cube that will convolve with a filter
            new_h=(imageRow-kerRow)//st+1, new_w=(imagrCol-kerCol)//st+1
    """

    chan, row, col = image.shape
    new_h = (row - kerRow) // st + 1
    new_w = (col - kerCol) // st + 1
    col = np.zeros([new_h * new_w, chan * kerRow * kerCol])

    for i in range(new_h):
        for j in range(new_w):
            patch = image[..., i * st:i * st + kerRow, j * st:j * st + kerCol]
            col[i * new_w + j, :] = np.reshape(patch, -1)
    return col

In [4]:
def mul(num):
    p = 1
    for m in num:
        p *= m
    # print('p=',p)
    return p

In [5]:
def col2im(ma, h_prime, w_prime, C):
    """
      Args:
      ma: (h_prime*w_prime*w,F) matrix, each col should be reshaped to C*h_prime*w_prime when C>0, or h_prime*w_prime when C = 0
      h_prime: reshaped filter height
      w_prime: reshaped filter width
      C: reshaped filter channel, if 0, reshape the filter to 2D, Otherwise reshape it to 3D
    Returns:
      if C == 0: (F,h_prime,w_prime) matrix
      Otherwise: (F,C,h_prime,w_prime) matrix
    """
    F = ma.shape[1]
    if (C == 1):
        out = np.zeros([F, h_prime, w_prime])
        for i in range(F):
            col = ma[:, i]
            out[i, :, :] = np.reshape(col, (h_prime, w_prime))
    else:
        out = np.zeros([F, C, h_prime, w_prime])
        for i in range(F):
            col = ma[:, i]
            out[i, :, :] = np.reshape(col, (C, h_prime, w_prime))

    return out

In [6]:
def conv(image, filt, b, st=1):
    imChan, imRow, imCol = image.shape
    inChan, outChan, kerRow, kerCol = filt.shape
    r = (imRow - 1) // st + 1
    c = (imCol - 1) // st + 1
    out = np.zeros((outChan, r, c))

    imPad = np.pad(image, ((0, 0), ((kerRow - 1) // 2, (kerRow - 1) // 2),
                           ((kerCol - 1) // 2, (kerCol - 1) // 2)))
    imMat = im2col(imPad, kerRow, kerCol, st)
    filtCol = np.reshape(filt, (outChan, -1))
    outCol = imMat.dot(filtCol.T) + b
    out = col2im(outCol, r, c, 1)
    return out

In [7]:
def act(layer):
    chan, row, col = layer.shape
    out = np.zeros((chan, row, col))

    for c in range(chan):
        for i in range(row):
            for j in range(col):
                if layer[c, i, j] >= 0:
                    out[c, i, j] = actWt[1] * layer[c, i, j]
                else:
                    out[c, i, j] = actWt[0] * layer[c, i, j]
    return out

In [8]:
def actInv(layer):
    chan, row, col = layer.shape
    out = np.zeros((chan, row, col))

    for c in range(chan):
        for i in range(row):
            for j in range(col):
                if layer[c, i, j] >= 0:
                    out[c, i, j] = layer[c, i, j] / actWt[1]
                else:
                    out[c, i, j] = layer[c, i, j] / actWt[0]
    return out

In [9]:
def actD(layer):
    chan, row, col = layer.shape
    out = np.zeros((chan, row, col))

    for c in range(chan):
        for i in range(row):
            for j in range(col):
                if layer[c, i, j] >= 0:
                    out[c, i, j] = actWt[1]
                else:
                    out[c, i, j] = actWt[0]
    return out

In [10]:
def trapInt(vals, ds):
    return ds / 2 * (np.sum(vals, axis=0) + np.sum(vals[1:-1], axis=0))
    #trapezoidal rule: (1/2)*ds*(f(x_0) + 2f(x_1) + 2f(x_2) + ... + 2f(x_n-1) + f(x_n))

In [11]:
def trapezoidIntegral(vals, ds):
    pass1 = tf.reduce_sum(vals, axis=0)
    pass2 = tf.reduce_sum(vals[1:-1], axis=0)
    return ds/2.0*(pass1+pass2)
def TFconv(image, filt, b):
    return tf.nn.convolution(input=image, filters=filt, padding="SAME") + b

In [42]:
def TFuSoln(raw_u0, ds, inFilt, pFilt, b):
    bzero = tf.zeros([inFilt.shape[4]])
    u0 = TFconv(raw_u0, inFilt[0], bzero)
    #for loop begins, h=0
    x0 = tf.nn.leaky_relu(u0, alpha=0.1)
    zInt0 = TFconv(x0, pFilt[0,0], b[0] / ((0+1)*ds))
    z0 = trapezoidIntegral(zInt0, ds)
    u1 = z0 + TFconv(raw_u0, inFilt[1], bzero)
    #h=1
    x1 = tf.nn.leaky_relu(u1, alpha=0.1)
    zInt1_0 = TFconv(x0, pFilt[0,1], b[1] / ((1+1)*ds)) #s=0
    zInt1_1 = TFconv(x1, pFilt[1,1], b[1] / ((1+1)*ds)) #s=1
    zInt1 = zInt1_0 + zInt1_1
    z1 = trapezoidIntegral(zInt1, ds)
    u2 = z1 + TFconv(raw_u0, inFilt[2], bzero)
    #h=2
    x2 = tf.nn.leaky_relu(u2, alpha=0.1)#
    zInt2_0 = TFconv(x0, pFilt[0,2], b[2] / ((2+1)*ds)) #s=0
    zInt2_1 = TFconv(x1, pFilt[1,2], b[2] / ((2+1)*ds)) #s=1
    zInt2_2 = TFconv(x2, pFilt[2,2], b[2] / ((2+1)*ds)) #s=2
    zInt2 = zInt2_0 + zInt2_1 + zInt2_2
    z2 = trapezoidIntegral(zInt2, ds)#
    u3 = z2 + TFconv(raw_u0, inFilt[3], bzero)
    #h=3
    x3 = tf.nn.leaky_relu(u3, alpha=0.1)
    zInt3_0 = TFconv(x0, pFilt[0,3], b[3] / ((3+1)*ds)) #s=0
    zInt3_1 = TFconv(x1, pFilt[1,3], b[3] / ((3+1)*ds)) #s=1
    zInt3_2 = TFconv(x2, pFilt[2,3], b[3] / ((3+1)*ds)) #s=2
    zInt3_3 = TFconv(x3, pFilt[3,3], b[3] / ((3+1)*ds)) #s=3
    zInt3 = zInt3_0 + zInt3_1 + zInt3_2 + zInt3_3
    z3 = trapezoidIntegral(zInt3, ds)
    u4 = z3 + TFconv(raw_u0, inFilt[4], bzero)
    #end loop
    x4 = tf.nn.leaky_relu(u4, alpha=0.1)
    
    print("z with h=0", z0)
    print("z with h=1", z1)
    print("z with h=2", z2)
    print("z with h=3", z3)
    
    x = tf.stack([x0,x1,x2,x3,x4])
    u = tf.stack([u0,u1,u2,u3,u4])
    
    return x,u


In [43]:
t0 = 4
t1 = t0+1
dt = 1.0/t0
input_shape = (1,5,5)
filter_shape = (1,1,3,3)



#np.random.seed(42)
actWt = [.1, 1] #leakyRelU: first number is slope left of origin, second number is slope right of origin

rng = np.random.default_rng(42)
uIn = rng.random(input_shape, dtype=np.float32)
iFilt = rng.random((t1,*filter_shape), dtype=np.float32)
paramFilt = rng.random((t1,t1,*filter_shape), dtype=np.float32)
biasVec = rng.random((t1,1), dtype=np.float32)

xOut, uOut = uSoln(uIn, dt, iFilt, paramFilt, biasVec)
#print(xOut)

h= 0
z[with h^]: 
 [[[0.59364062 0.7408913  0.74378842 0.60450126 0.42866077]
  [0.84826643 1.06295896 1.14045107 0.92155288 0.59152663]
  [0.9238679  1.33525803 1.38599361 1.17783711 0.77465796]
  [0.86133847 1.20479145 1.34143498 1.20356113 0.79090608]
  [0.61344388 0.90366097 0.97861385 0.97160753 0.71915292]]]
h= 1
z[with h^]: 
 [[[1.2397398  1.99178296 2.10615177 1.86093511 1.339403  ]
  [1.96901632 3.19553128 3.54247692 3.05614    2.10418588]
  [2.25595972 3.80578165 4.24106912 3.83901322 2.72757409]
  [2.04554903 3.53273044 4.02521385 3.78281474 2.71170038]
  [1.52757738 2.48901326 2.85933173 2.91272486 2.13996837]]]
h= 2
z[with h^]: 
 [[[ 2.54508911  4.33701102  4.61785467  4.10415892  2.62105438]
  [ 4.4840407   7.52447741  8.31580029  7.1220817   4.60002032]
  [ 5.49807065  9.06302506 10.21556968  9.14167307  6.02795693]
  [ 4.87158335  8.23812493  9.41576885  8.74603293  5.86528613]
  [ 3.35916209  5.5982222   6.45742824  6.33035723  4.07914058]]]
h= 3
z[with h^]: 
 [[[ 5.30

In [45]:
#TFuIn = tf.transpose()
TFuIn = tf.expand_dims(tf.transpose(tf.convert_to_tensor(uIn, dtype=tf.float32), perm=[1,2,0]), 0)
TFiFilt = tf.transpose(tf.convert_to_tensor(iFilt, dtype=tf.float32), perm=[0,3,4,1,2])
TFpFilt = tf.transpose(tf.convert_to_tensor(paramFilt, dtype=tf.float32), perm=[0,1,4,5,2,3])
TFbiasVec = tf.convert_to_tensor(biasVec, dtype=tf.float32)

TFxOut, TFuOut = TFuSoln(TFuIn, dt, TFiFilt, TFpFilt, TFbiasVec)
#print(tf.transpose(TFxOut, perm=[4,0,1,2,3]))


z with h=0 tf.Tensor(
[[[0.59364057]
  [0.74089134]
  [0.7437884 ]
  [0.60450125]
  [0.42866078]]

 [[0.8482664 ]
  [1.062959  ]
  [1.1404511 ]
  [0.9215529 ]
  [0.5915266 ]]

 [[0.9238679 ]
  [1.335258  ]
  [1.3859936 ]
  [1.1778371 ]
  [0.77465796]]

 [[0.8613385 ]
  [1.2047915 ]
  [1.3414351 ]
  [1.2035611 ]
  [0.7909061 ]]

 [[0.61344385]
  [0.90366095]
  [0.97861385]
  [0.9716075 ]
  [0.7191529 ]]], shape=(5, 5, 1), dtype=float32)
z with h=1 tf.Tensor(
[[[1.2397398]
  [1.9917828]
  [2.1061518]
  [1.860935 ]
  [1.3394029]]

 [[1.9690162]
  [3.1955314]
  [3.542477 ]
  [3.0561402]
  [2.104186 ]]

 [[2.2559595]
  [3.8057816]
  [4.241069 ]
  [3.8390136]
  [2.7275739]]

 [[2.045549 ]
  [3.5327303]
  [4.0252137]
  [3.7828147]
  [2.7117002]]

 [[1.5275774]
  [2.4890132]
  [2.8593316]
  [2.9127247]
  [2.1399684]]], shape=(5, 5, 1), dtype=float32)
z with h=2 tf.Tensor(
[[[1.9021738]
  [3.2220302]
  [3.479662 ]
  [3.0603938]
  [1.9655857]]

 [[3.491907 ]
  [5.805031 ]
  [6.429448 ]
  [5.4741